In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [2]:
tool = TavilySearchResults(max_results=2)

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [6]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
import os
prompt = """你是一个聪明的研究助手。使用搜索引擎查找信息。
你可以进行多次调用（一起或依次进行）。
只有在确定想要什么时才查找信息。
如果你在提出后续问题之前需要查找一些信息，你可以这样做！
"""
api_key = os.environ.get("MY_API_KEY")
base_url = os.environ.get("OPENAI_API_BASE")
model = ChatOpenAI(api_key=api_key, base_url=base_url,model="gpt-4-turbo") 
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [8]:
messages = [HumanMessage(content="西安今天的天气如何?")]

In [9]:
thread = {"configurable": {"thread_id": "1"}}

In [10]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mWP9wiim35FvDNOIfN4HAzMs', 'function': {'arguments': '{"query":"今天西安的天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 178, 'total_tokens': 203}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6226e07b-63bf-4f71-9b40-244de66a1587-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '今天西安的天气'}, 'id': 'call_mWP9wiim35FvDNOIfN4HAzMs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 178, 'output_tokens': 25, 'total_tokens': 203})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': '今天西安的天气'}, 'id': 'call_mWP9wiim35FvDNOIfN4HAzMs', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content="[{'url': 'https://wx.weather.com.cn/mweather/101110101.shtml', 'content': '西安天气预报，及时准确发布中央气象台天气信息，便捷查询西安今日天

In [11]:
messages = [HumanMessage(content="宝鸡今天的天气如何?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JgmpHK97CdAKHyBocNtDMwKM', 'function': {'arguments': '{"query":"今天宝鸡的天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 686, 'total_tokens': 714}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-63ab40d2-a95b-42a9-a071-20e38f95c633-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '今天宝鸡的天气'}, 'id': 'call_JgmpHK97CdAKHyBocNtDMwKM', 'type': 'tool_call'}], usage_metadata={'input_tokens': 686, 'output_tokens': 28, 'total_tokens': 714})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': '今天宝鸡的天气'}, 'id': 'call_JgmpHK97CdAKHyBocNtDMwKM', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content="[{'url': 'https://www.tianqi.com/baoji/', 'content': '天气网提供宝鸡天气预报15天,30天,今日天气,明天

In [12]:
messages = [HumanMessage(content="这两个城市哪个更暖和一些?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iPdmukjcJPEDVvhy1QH4sqbd', 'function': {'arguments': '{"query": "西安今天的实时温度"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_DRINnoQQVqKy99nxoGR3gxqP', 'function': {'arguments': '{"query": "宝鸡今天的实时温度"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 1236, 'total_tokens': 1308}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f59fdf19-d27b-4295-aa38-88429a4fda97-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '西安今天的实时温度'}, 'id': 'call_iPdmukjcJPEDVvhy1QH4sqbd', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': '宝鸡今天的实时温度'}, 'id': 'call_DRINnoQQVqKy99nxoGR3gxqP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1236, 'output_tokens': 72, 'tot

In [13]:
messages = [HumanMessage(content="这两个城市哪个更暖和一些?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='请告诉我你想比较的两个城市的名字。', response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 182, 'total_tokens': 203}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_0993c4a4c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-c91b447c-eea4-41c3-977f-e38eb1fd9eaa-0', usage_metadata={'input_tokens': 182, 'output_tokens': 21, 'total_tokens': 203})]}


In [17]:
!pip install aiosqlite

In [18]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [ ]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chain_stream":
        print(event["data"]["chunk"]["messages"])
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")